# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2020 &ndash; Week 0 &ndash; ETH Zurich</center>

## 1. Jupyter Basics

Welcome to this Jupyter notebook. Jupyter is a web-based open-source tool based on Python that allows you to run Python (and other types of) code, visualize and discuss results, and organize everything into notebooks like this one. We use Google Colab, but you can also [install your own server}](https://jupyter.readthedocs.io/en/latest/install/notebook-classic.html).

A notebook is organized in cells. Cells of this notebook contain Python code (but other cell types exists). To run a cell, select it, then press ctrl+enter. Try it out!

In [ ]:
print("Hello World")

By default, the last expression is printed. Like this:

In [ ]:
maxX = 10
[x * x for x in range(maxX)]

You can also edit the text. Just double-click on a cell. It's made with markdown code. After you are done editing, press ctrl+enter

We will do about half of the exercises with Jupyter. You will learn most things as we go. The notebook you are seing is your copy stored on your account (including the output of the cells). Write your answers inline and save regularly. (Also save a copy somewhere else from time to time &ndash; this service is still beta.)

### Read more:
* [Jupyter](http://jupyter.org/)
* [Learn Python](http://www.learnpython.org)
* [Python documentation](https://docs.python.org/3/)

## 2. Bash Scripts

Code blocks by default are executed using a python interpreter (for a python notebook, such as this one).

Other languages can be used with annotations.
For instance, a code block can be converted into a bash code block using ```%%bash``` at the beginning:

In [ ]:
%%bash
echo "Test File" > test_file
cat test_file

Note: do not expect files you write into this file system to be durable. The sandboxed environment may be reset and data lost when notebook is closed. The notebook files themselves are durable though.

Bash commands can be also inlined using exclamation mark (```!```) infront of the bash line

In [ ]:
print("First I modify the file.")

!echo "Hello World" > test_file

print("Then I check its content:")
print()

!cat test_file

## 3. Extensions

As part of this course you will use Jupyter to interact with various systems and interfaces (e.g. SQL, Map Reduce, Spark). To use these, you will need to install certain Python and Jupyter extensions. Note, that as like the file system the extensions will not be durable and need to be rerun when the notebook server is restarted.

#### 3.1. Setting up a SQL connection

As part of preparation for the next week's exercise, let's setup a connection to a sample relational database (hosted from this course's Azure account).

First set the access variables (make sure you execute the next code block by running ctrl+enter).

In [ ]:
server='ethbigdata2020.postgres.database.azure.com'
user='student@ethbigdata2020'
password='BigData2020'
database='poker.stackexchange.com'
connection_string=f'postgresql://{user}:{password}@{server}:5432/{database}?sslmode=require'

#### 3.2. Running a SQL Query
Then we run a first query against our server (following [this tutorial](https://docs.microsoft.com/en-us/azure/postgresql/quickstart-create-server-database-portal) from the Azure website). This should print the version information of the SQL server.

In [ ]:
import sqlalchemy

engine = sqlalchemy.create_engine(connection_string)
print(engine.execute('SELECT version()').fetchall())

#### 3.3. Inlining SQL 
It is also possible to inline SQL code (using [SQL "magic"](https://github.com/catherinedevlin/ipython-sql)).

We can now load the extension and establish a connection to our database from above. Run the following cell and make sure the output says *"Connected: &lt;connection string&gt;"*.

In [ ]:
%reload_ext sql
%sql $connection_string

Now we can use the ```%sql``` and ```%%sql``` magic words to run SQL directly. ```%%sql``` makes a cell a SQL cell. A SQL cell can run an arbitrary number of SQL statements and displays the result of the last one of them.

Let's see the version number again:

In [ ]:
%%sql 
SELECT version();

The ```%sql``` magic words lets us run SQL statements in a regular cell. Again, the result of the last statement is displayed.

In [ ]:
print("Run a query!")
%sql SELECT Id, DisplayName FROM Users ORDER BY Id LIMIT 10;

#### 3.4. Plotting results
Matplotlib can also be used to plot results. Next is a plot of a sample query that finds the number of created users per year.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

result = %sql SELECT DATE_PART('year', CreationDate) as CreationYear, COUNT(*) as Count \
                FROM Users \
                GROUP BY DATE_PART('year', CreationDate) \
                ORDER BY DATE_PART('year', CreationDate) ASC;
            
# Print the result in tabular form
print(result)
            
# Convert the result to a Pandas data frame
df = result.DataFrame()

# Extract x and y values for a plot
x = df['creationyear'].tolist()
y = df['count'].tolist()

# Plot the distribution of registrations per year
fig, ax = plt.subplots()
ax.bar(range(len(df.index)), y, tick_label=[int(i) for i in x], align='center')
ax.set_xlabel('Creation Year')
ax.set_ylabel('Number of Users')